In [1]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import torch
from diffusers import StableDiffusionPipeline,DDIMScheduler
import torch.nn.functional as nnf
import numpy as np
import abc
import ptp_utils as ptp_utils
import seq_aligner

In [2]:
LOW_RESOURCE = False 
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 7.5
MAX_NUM_WORDS = 77
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
ldm_stable = StableDiffusionPipeline.from_pretrained("sd-legacy/stable-diffusion-v1-5", torch_dtype=torch.bfloat16).to(device)
from diffusers import DDIMScheduler
ldm_stable.scheduler = DDIMScheduler.from_config(ldm_stable.scheduler.config)
tokenizer = ldm_stable.tokenizer

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


## Prompt-to-Prompt Attnetion Controllers
Our main logic is implemented in the `forward` call in an `AttentionControl` object.
The forward is called in each attention layer of the diffusion model and it can modify the input attnetion weights `attn`.

`is_cross`, `place_in_unet in ("down", "mid", "up")`, `AttentionControl.cur_step` help us track the exact attention layer and timestamp during the diffusion iference.


In [3]:
class LocalBlend:

    def __call__(self, x_t, attention_store):
        k = 1
        maps = attention_store["down_cross"][2:4] + attention_store["up_cross"][:3]
        maps = [item.reshape(self.alpha_layers.shape[0], -1, 1, 16, 16, MAX_NUM_WORDS) for item in maps]
        maps = torch.cat(maps, dim=1)
        maps = (maps * self.alpha_layers).sum(-1).mean(1)
        mask = nnf.max_pool2d(maps, (k * 2 + 1, k * 2 +1), (1, 1), padding=(k, k))
        mask = nnf.interpolate(mask, size=(x_t.shape[2:]))
        mask = mask / mask.max(2, keepdims=True)[0].max(3, keepdims=True)[0]
        mask = mask.gt(self.threshold)
        mask = (mask[:1] + mask[1:]).float()
        x_t = x_t[:1] + mask * (x_t - x_t[:1])
        return x_t
       
    def __init__(self, prompts: List[str], words: [List[List[str]]], threshold=.3):
        alpha_layers = torch.zeros(len(prompts),  1, 1, 1, 1, MAX_NUM_WORDS)
        for i, (prompt, words_) in enumerate(zip(prompts, words)):
            if type(words_) is str:
                words_ = [words_]
            for word in words_:
                ind = ptp_utils.get_word_inds(prompt, word, tokenizer)
                alpha_layers[i, :, :, :, :, ind] = 1
        self.alpha_layers = alpha_layers.to(device)
        self.threshold = threshold


class AttentionControl(abc.ABC):
    
    def step_callback(self, x_t):
        return x_t
    
    def between_steps(self):
        return
    
    @property
    def num_uncond_att_layers(self):
        return self.num_att_layers if LOW_RESOURCE else 0
    
    @abc.abstractmethod
    def forward (self, attn, is_cross: bool, place_in_unet: str):
        raise NotImplementedError

    def __call__(self, attn, is_cross: bool, place_in_unet: str):
        if self.cur_att_layer >= self.num_uncond_att_layers:
            if LOW_RESOURCE:
                attn = self.forward(attn, is_cross, place_in_unet)
            else:
                # 修正: 避免 in-place 操作，使用 clone() 和 cat()
                h = attn.shape[0]
                attn_uncond = attn[:h // 2]
                attn_cond = self.forward(attn[h // 2:].clone(), is_cross, place_in_unet)
                attn = torch.cat([attn_uncond, attn_cond], dim=0)
               
        self.cur_att_layer += 1
        if self.cur_att_layer == self.num_att_layers + self.num_uncond_att_layers:
            self.cur_att_layer = 0
            self.cur_step += 1
            self.between_steps()
        return attn
    
    def reset(self):
        self.cur_step = 0
        self.cur_att_layer = 0

    def __init__(self):
        self.cur_step = 0
        self.num_att_layers = -1
        self.cur_att_layer = 0

class EmptyControl(AttentionControl):
    
    def forward (self, attn, is_cross: bool, place_in_unet: str):
        return attn
 
class AttentionStore(AttentionControl):

    @staticmethod
    def get_empty_store():
        return {"down_cross": [], "mid_cross": [], "up_cross": [],
                "down_self": [],  "mid_self": [],  "up_self": []}

    def forward(self, attn, is_cross: bool, place_in_unet: str):
        key = f"{place_in_unet}_{'cross' if is_cross else 'self'}"
        if attn.shape[1] <= 32 ** 2:  # avoid memory overhead
            self.step_store[key].append(attn)
        return attn

    def between_steps(self):
        if len(self.attention_store) == 0:
            self.attention_store = self.step_store
        else:
            for key in self.attention_store:
                for i in range(len(self.attention_store[key])):
                    self.attention_store[key][i] += self.step_store[key][i]
        self.step_store = self.get_empty_store()

    def get_average_attention(self):
        average_attention = {key: [item / self.cur_step for item in self.attention_store[key]] for key in self.attention_store}
        return average_attention


    def reset(self):
        super(AttentionStore, self).reset()
        self.step_store = self.get_empty_store()
        self.attention_store = {}

    def __init__(self):
        super(AttentionStore, self).__init__()
        self.step_store = self.get_empty_store()
        self.attention_store = {}
    
        
class AttentionControlEdit(AttentionStore, abc.ABC):
    
    def step_callback(self, x_t):
        if self.local_blend is not None:
            x_t = self.local_blend(x_t, self.attention_store)
        return x_t
        
    def replace_self_attention(self, attn_base, att_replace):
        if att_replace.shape[2] <= 16 ** 2:
            return attn_base.unsqueeze(0).expand(att_replace.shape[0], *attn_base.shape)
        else:
            return att_replace
    
    @abc.abstractmethod
    def replace_cross_attention(self, attn_base, att_replace):
        raise NotImplementedError
    
    def forward(self, attn, is_cross: bool, place_in_unet: str):
        super(AttentionControlEdit, self).forward(attn, is_cross, place_in_unet)
        if is_cross or (self.num_self_replace[0] <= self.cur_step < self.num_self_replace[1]):
            h = attn.shape[0] // (self.batch_size)
            attn = attn.reshape(self.batch_size, h, *attn.shape[1:])
            attn_base, attn_repalce = attn[0], attn[1:]
            if is_cross:
                # 轉換 alpha_words 到與 attn 相同的 dtype
                alpha_words = self.cross_replace_alpha[self.cur_step].to(attn.dtype)
                attn_repalce_new = self.replace_cross_attention(attn_base, attn_repalce) * alpha_words + (1 - alpha_words) * attn_repalce
                attn = torch.cat([attn_base.unsqueeze(0), attn_repalce_new], dim=0)
            else:
                attn = torch.cat([attn_base.unsqueeze(0), self.replace_self_attention(attn_base, attn_repalce)], dim=0)
            attn = attn.reshape(self.batch_size * h, *attn.shape[2:])
        return attn
    
    def __init__(self, prompts, num_steps: int,
                 cross_replace_steps: Union[float, Tuple[float, float], Dict[str, Tuple[float, float]]],
                 self_replace_steps: Union[float, Tuple[float, float]],
                 local_blend: Optional[LocalBlend]):
        super(AttentionControlEdit, self).__init__()
        self.batch_size = len(prompts)
        self.cross_replace_alpha = ptp_utils.get_time_words_attention_alpha(prompts, num_steps, cross_replace_steps, tokenizer).to(device)
        if type(self_replace_steps) is float:
            self_replace_steps = 0, self_replace_steps
        self.num_self_replace = int(num_steps * self_replace_steps[0]), int(num_steps * self_replace_steps[1])
        self.local_blend = local_blend

class AttentionRefine(AttentionControlEdit):

    def replace_cross_attention(self, attn_base, att_replace):
        attn_base_replace = attn_base[:, :, self.mapper].permute(2, 0, 1, 3)
        # 轉換 alphas 到與 attn 相同的 dtype
        alphas = self.alphas.to(attn_base.dtype)
        attn_replace = attn_base_replace * alphas + att_replace * (1 - alphas)
        return attn_replace

    def __init__(self, prompts, num_steps: int, cross_replace_steps: float, self_replace_steps: float,
                 local_blend: Optional[LocalBlend] = None):
        super(AttentionRefine, self).__init__(prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend)
        self.mapper, alphas = seq_aligner.get_refinement_mapper(prompts, tokenizer)
        self.mapper, alphas = self.mapper.to(device), alphas.to(device)
        self.alphas = alphas.reshape(alphas.shape[0], 1, 1, alphas.shape[1])






## 實驗開始前的設定修改

In [ ]:
from experiment_config import TrainConfig,ExperimentConfig
import glob
import os
cfg = ExperimentConfig(
    exp_dir="20251203_014400",
    # 這邊是實驗的名稱會存在experiments/這個資料夾底下 new代表會依照實驗時間自動建立子資料夾，不是new的話就會直接存在experiments/{exp_dir}底下
    base_dir="experiments/",
    source_image_path="/home/ksp0108/workspace/HW/video_gen/prompt-to-prompt/dataset/test_1201(pair_data)/B_05.png",
    #BEFORE圖片路徑
    target_image_path="/home/ksp0108/workspace/HW/video_gen/prompt-to-prompt/dataset/test_1201(pair_data)/A_05.png",
    #AFTER圖片路徑
    test_image_pattern= "/home/ksp0108/workspace/HW/video_gen/prompt-to-prompt/dataset/test_1130(single_data)/*.png",
    #這邊是glob.glob的pattern會把符合的圖片都拿來做測試
    low_resource= False,
    #不知道什麼意思就不要動它，原本P2P留下的
    num_diffusion_steps= 50,
    # 這邊是sd的diffusion steps數量，預設就50不要動




    # 以下為實驗可調整的參數
    option_guidance_scale= [1, 3.5, 7.5],
    # guidance_scale 可以 1 3.5 7.5去比較效果
    option_cross_replace_step= [[0.2,1.0]], #等價於論文的 tau =0.8
    #這邊每一個serach instance是一個list第一項是開始交換的tau比例第二項是結束交換的tau比例
    #論文中的if tau(0.7)>t -> do differential attention control，T是從1開始倒數到0
    #但我們這邊時做事候，是從0開始數到1所以要做交換的tau比例要做反轉，例如論文中tau=0.7代表我們這裡t從0.3,1.0要做交換
    option_encoded_emb= [False],
    # sd 1.5處理文字的流程如下: text->tokenizer->embedding->text encoder
    # 如果是encoded_emb=False，代表我們訓練的是進入text encoder前的embedding向量(這個是paper主要實驗的設定包含text-inversion這篇也是這樣做)
    # 如果是encoded_emb=True，代表我們訓練的是進入text encoder後的embedding向量
    option_self_replace_step= [0.],
    # 這邊論文對self的交換沒有寫得很詳細跳過這個部分
    coarse_description= "a watercolor painting",
    #coarse_description是用來控制風格的描述詞初始化用的


    lr= 0.001,
    optimizer_cls= torch.optim.AdamW,
    num_epochs= 40,
    save_interval= 1,
    beta_weighting= True,
    test_epochs= [0, 5, 10, 20, 30, 39]

)

## 搜尋的參數組合

In [5]:
# Generate all TrainConfig objects from ExperimentConfig
train_configs = cfg.generate_train_configs()

# Filter out already completed experiments
filtered_configs = []
for train_cfg in train_configs:
    save_image_dir = train_cfg.get_save_dir()
    if os.path.exists(os.path.join(save_image_dir, "final.pt")):
        print(f"Skipping {save_image_dir} since final.pt exists")
        continue
    filtered_configs.append(train_cfg)
train_configs = filtered_configs

print(f"Running {len(train_configs)} experiments")
if train_configs:
    exp_dir = train_configs[0].exp_dir
    print(f"Experiment directory: {exp_dir}")
    # Save experiment config to experiment directory
    cfg_save_path = os.path.join(exp_dir, "config.json")
    cfg.save(cfg_save_path)
    print(f"Config saved to {cfg_save_path}")

Skipping experiments/20251203_014400/cross_replace=[0.2, 1.0],self_replace=0.0,encoded=False,guidance_scale=1, since final.pt exists
Skipping experiments/20251203_014400/cross_replace=[0.2, 1.0],self_replace=0.0,encoded=False,guidance_scale=3.5, since final.pt exists
Skipping experiments/20251203_014400/cross_replace=[0.2, 1.0],self_replace=0.0,encoded=False,guidance_scale=7.5, since final.pt exists
Running 0 experiments


## 對訓練影像作前處理(DDIM inversion)

In [6]:
# Load images using paths from config
from inversion import invert
from PIL import Image


source_image = Image.open(cfg.source_image_path).convert("RGB").resize((512, 512))
target_image = Image.open(cfg.target_image_path).convert("RGB").resize((512, 512))

# Convert to latents
before_latent = ptp_utils.image2latent(ldm_stable.vae, np.array(source_image).reshape(1, 512, 512, 3))
after_latent = ptp_utils.image2latent(ldm_stable.vae, np.array(target_image).reshape(1, 512, 512, 3))

# Do inversion
noised_before_latent = invert(
    ldm_stable,
    before_latent,
    prompt="",
    guidance_scale=1,
    num_inference_steps=cfg.num_diffusion_steps,
    device=device,
)

print("Images loaded and inverted")

Registered 16 cross attention layers.


  0%|          | 0/50 [00:00<?, ?it/s]

Images loaded and inverted


## 開始訓練

In [7]:
for train_cfg in train_configs:
    save_image_dir = train_cfg.get_save_dir()
    os.makedirs(save_image_dir, exist_ok=True)
    
    # Save TrainConfig to experiment folder
    train_cfg_path = os.path.join(save_image_dir, "train_config.json")
    train_cfg.save(train_cfg_path)
    print(f"TrainConfig saved to {train_cfg_path}")
    
    # Create attention controller
    controller = AttentionRefine(
        prompts=["", train_cfg.coarse_description],
        num_steps=train_cfg.num_diffusion_steps,
        cross_replace_steps=train_cfg.cross_replace_step,
        self_replace_steps=train_cfg.self_replace_step,
    )
    
    # Setup for training
    torch.autograd.set_detect_anomaly(True)
    ldm_stable.vae.requires_grad_(False)
    ldm_stable.unet.requires_grad_(False)
    ldm_stable.text_encoder.requires_grad_(False)
    ldm_stable.scheduler = DDIMScheduler.from_pretrained("sd-legacy/stable-diffusion-v1-5", subfolder="scheduler")
    ldm_stable.scheduler.set_timesteps(train_cfg.num_diffusion_steps)
    
    # Select training function based on config
    train_fn = ptp_utils.train_text_embedding_ldm_stable if train_cfg.encoded_emb else ptp_utils.train_text_embedding_ldm_stable_with_out_encode
    
    # Train
    learned_emb = train_fn(
        ldm_stable,
        train_cfg.coarse_description,
        controller,
        noised_before_latent,
        after_latent,
        num_steps=train_cfg.num_diffusion_steps,
        epoch=train_cfg.num_epochs,
        guidance_scale=train_cfg.guidance_scale,
        lr=train_cfg.lr,
        optimizer_cls=train_cfg.optimizer_cls,
        save_interval=train_cfg.save_interval,
        save_image_dir=save_image_dir,
        beta_weighting=train_cfg.beta_weighting
    )

TrainConfig saved to experiments/20251203_014400/cross_replace=[0.2, 1.0],self_replace=0.0,encoded=False,guidance_scale=3.5,/train_config.json
Registered 16 cross attention layers.


/home/ksp0108/workspace/HW/video_gen/prompt-to-prompt/ptp_utils.py:676: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  X_t=latent.expand(2, model.unet.in_channels, height // 8, width // 8).to(model.device)


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 0.1348


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch 38, Loss: 0.0834


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 39, Loss: 0.0886


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 40, Loss: 0.0807


## 讀取訓練後的模型，和配置進行測試

In [10]:
from experiment_config import get_or_create_exp_dir,ExperimentConfig
import os
target_exp_dir = get_or_create_exp_dir(
    base_dir="experiments/",
    exp_dir=None #這邊可以指定要讀取哪一個實驗的資料夾名稱(如:20251203_005726)，如果為None則會挑最新的一次
)#experiments/20251203_005726
target_exp_id=target_exp_dir.split("/")[-1]#20251203_005726
cfg = ExperimentConfig().load(os.path.join(target_exp_dir, "config.json"))
test_configs=cfg.generate_train_configs()
all_test_image = glob.glob(cfg.test_image_pattern)
print(f"Testing experiments in: {test_configs[0].exp_dir if test_configs else 'N/A'}")
print(f"Test images: {len(all_test_image)} files")

Using existing experiment directory: experiments/20251203_014400
Testing experiments in: experiments/20251203_014400
Test images: 11 files


In [ ]:
from image_utils import ImageGrid
# Define column labels based on test epochs
col_labels = ["Source", "Epoch0"] + [f"Epoch{e}" for e in cfg.test_epochs[1:]]

# Run inference on test images
for test_cfg in test_configs:
    exp_dir = test_cfg.exp_dir
    
    # Create ImageGrid with labels
    row_labels = [os.path.basename(p).replace('.jpg', '') for p in all_test_image]
    image_grid = ImageGrid(row_labels=row_labels, col_labels=col_labels)
    
    for image_path in all_test_image:
        before_image = Image.open(image_path).convert("RGB").resize((512, 512))
        before_latent = ptp_utils.image2latent(ldm_stable.vae, np.array(before_image).reshape(1, 512, 512, 3))
        
        noised_before_latent = invert(
            ldm_stable,
            before_latent,
            prompt="",
            guidance_scale=1,
            num_inference_steps=test_cfg.num_diffusion_steps,
            device=device,
        )
        
        for e in test_cfg.test_epochs:
            learned_emb_path = os.path.join(exp_dir, test_cfg.exp_name, f"epoch_{e}.pt")
            learned_emb = torch.load(learned_emb_path).to(device)
            
            # Create controller
            controller = AttentionRefine(
                prompts=["", test_cfg.coarse_description],
                num_steps=test_cfg.num_diffusion_steps,
                cross_replace_steps=test_cfg.cross_replace_step,
                self_replace_steps=0.0,
            )
            
            images, x_t = ptp_utils.text2image_ldm_stable_with_learned_embedding(
                ldm_stable,
                learned_emb=learned_emb,
                controller=controller,
                latent=noised_before_latent,
                num_inference_steps=test_cfg.num_diffusion_steps,
                guidance_scale=test_cfg.guidance_scale,
                low_resource=test_cfg.low_resource
            )
            
            if e == test_cfg.test_epochs[0]:
                # Add source image and first epoch result
                image_grid.add_image([images[0], images[1]])
            else:
                image_grid.add_image(images[1])
    
    # Save with row and column labels
    save_path = os.path.join(exp_dir, test_cfg.exp_name, "test_image1.png")
    image_grid.save(save_path, num_rows=len(all_test_image))
    print(f"Saved images to {save_path}")

Registered 16 cross attention layers.


  0%|          | 0/50 [00:00<?, ?it/s]

Registered 16 cross attention layers.


/home/ksp0108/workspace/HW/video_gen/prompt-to-prompt/ptp_utils.py:240: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  latents = latent.expand(batch_size,  model.unet.in_channels, height // 8, width // 8).to(model.device)


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Registered 16 cross attention layers.


  0%|          | 0/50 [00:00<?, ?it/s]

Registered 16 cross attention layers.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
result_configs = cfg.generate_train_configs()

# Copy results to comparison folder within the experiment directory
if result_configs:
    exp_dir = result_configs[0].exp_dir
    comparison_dir = os.path.join(exp_dir, "all_option_combinations")
    os.makedirs(comparison_dir, exist_ok=True)
    
    for result_cfg in result_configs:
        test_image_path = os.path.join(exp_dir, result_cfg.exp_name, "test_image1.png")
        safe_name = result_cfg.exp_name.replace('(', '').replace(')', '').replace(',', '_').replace('=', '_')
        dest_path = os.path.join(comparison_dir, f"{safe_name}_test_image1.png")
        
        if os.path.exists(test_image_path):
            os.system(f"cp '{test_image_path}' '{dest_path}'")
    
    print(f"Results copied to {comparison_dir}/")

Results copied to experiments/20251203_010454/all_option_combinations/
